# Research

Research is a framework to provide multiple experiments with pipelines.

In [1]:
import tensorflow as tf
import dill

import matplotlib.pyplot as plt
%matplotlib inline

#sys.path.append("../..")
from dataset import Pipeline, B, C, V
from dataset.opensets import MNIST
from dataset.models.tf import VGG7, VGG16
from dataset.research import Research, Option

C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
BATCH_SIZE=64

In [3]:
model_config={
    'session/config': tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.5)),
    'inputs': dict(images={'shape': (28, 28, 1)},
                   labels={'classes': 10, 'transform': 'ohe', 'name': 'targets'}),
    'input_block/inputs': 'images',
    'body/block/layout': C('layout'),
    'output/ops': 'accuracy'
}

In [4]:
train_template = (Pipeline()
            .init_variable('loss', init_on_each_run=list)
            .init_variable('accuracy', init_on_each_run=list)
            .init_model('dynamic', C('model'), 'conv', config=model_config)
            .to_array()
            .train_model('conv', 
                         fetches=['loss', 'output_accuracy'], 
                         feed_dict={'images': B('images'), 'labels': B('labels')},
                         save_to=[V('loss'), V('accuracy')], mode='a')
)

In [5]:
test_template = (Pipeline()
            .init_variable('accuracy', init_on_each_run=list)
            .import_model('conv', C('import_from'))
            .to_array()
            .predict_model('conv', 
                         fetches=['output_accuracy'], 
                         feed_dict={'images': B('images'), 'labels': B('labels')},
                         save_to=[V('accuracy')], mode='a')
)

In [6]:
mnist = MNIST()

DownloadingDownloadingDownloadingDownloading   http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gzhttp://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gzhttp://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz



ExtractingExtractingExtractingExtracting    C:\Users\kozhevin\AppData\Local\Temp\train-images-idx3-ubyte.gzC:\Users\kozhevin\AppData\Local\Temp\t10k-labels-idx1-ubyte.gzC:\Users\kozhevin\AppData\Local\Temp\t10k-images-idx3-ubyte.gzC:\Users\kozhevin\AppData\Local\Temp\train-labels-idx1-ubyte.gz





In [7]:
train_root = mnist.train.p.run(BATCH_SIZE, shuffle=True, n_epochs=1, lazy=True)
test_root = mnist.test.p.run(BATCH_SIZE, shuffle=True, n_epochs=1, lazy=True)

In [8]:
grid = Option('layout', ['cna', 'can']) * Option('model', [VGG7, VGG16])

In [9]:
def get_accuracy(experiment, iteration, pipeline):
    import numpy as np
    acc = pipeline.get_variable('accuracy')
    print(pipeline, acc)
    return np.mean(acc)

In [10]:
def plot_loss(experiment, iteration, pipeline):
    import matplotlib.pyplot as plt
    loss = pipeline.get_variable('loss')
    plt.plot(loss)
    plt.savefig(experiment.config['path']+'/train_loss_{:03d}'.format(iteration))
    plt.close()

In [11]:
mr = (
    Research()
    .pipeline(train_root, train_template, variables=['loss'], name='train')
    .pipeline(test_root, test_template, variables='accuracy', name='test', 
              run=True, execute_for=10, dump_for=10, import_from='train')
    .function(get_accuracy, returns='accuracy', name='test_accuracy', execute_for=10, dump_for=20, pipeline='test')
    .function(plot_loss, name='plot_loss', execute_for=10, pipeline='train')
    .grid(grid)
     )

In [12]:
mr.run(n_reps=6, n_iters=100, n_workers=1, branches=2, name='my_research', progress_bar=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [07:50<00:00, 39.17s/it]


In [13]:
import dill
with open('my_research/results/layout_can-model_VGG7/0/test_accuracy', 'rb') as file:
    res = dill.load(file)

In [14]:
res

{'accuracy': [0.125,
  0.09375,
  0.19791667,
  0.23828125,
  0.26875,
  0.25,
  0.26785713,
  0.28710938,
  0.3298611,
  0.36875],
 'iterations': [9, 19, 29, 39, 49, 59, 69, 79, 89, 99]}